# Elliptic Curve Diffie Hellman

In [32]:
INFINITY = None


class EllipticCurve:
    def __init__(self, a, b, p, G, n, h):
        self.a = a
        self.b = b
        self.p = p
        self.G = G
        self.n = n
        self.h = h

    def add(self, P, Q):
        if P == INFINITY:
            return Q
        if Q == INFINITY:
            return P

        (x1, y1) = P
        (x2, y2) = Q

        # If we add points symmetrical with respect to Y-axis
        if self.equal_modp(x1, x2) and self.equal_modp(y1, -y2):
            return INFINITY

        # Calculation slope for:
        # Point doubling
        if self.equal_modp(x1, x2) and self.equal_modp(y1, y2):
            u = self.reduce_modp((3 * x1 * x1 + self.a) * self.inverse_modp(2 * y1))
        # Point addition
        else:
            u = self.reduce_modp((y1 - y2) * self.inverse_modp(x1 - x2))

        v = self.reduce_modp(y1 - u * x1)
        x3 = self.reduce_modp(u * u - x1 - x2)
        y3 = self.reduce_modp(-u * x3 - v)
        return x3, y3

    def multiply(self, k, P):
        # Double and add algorithm from least significant bit to most significant bit
        Q = INFINITY
        while k != 0:
            if k & 1 != 0:
                Q = self.add(Q, P)
            P = self.add(P, P)
            k >>= 1
        return Q

    def reduce_modp(self, x):
        return x % self.p

    def equal_modp(self, x, y):
        return self.reduce_modp(x - y) == 0

    def inverse_modp(self, x):
        if self.reduce_modp(x) == 0:
            return None
        return pow(x, self.p - 2, self.p)#From Fermat's therorem supposing p is prime

    def is_point_on_curve(self, x, y):
        return self.equal_modp(y * y, x * x * x + self.a * x + self.b)

First, Alice and Bob choose an elliptic curve. Let's say it's NIST P-224 which has these parameters :

In [33]:
p = 23
a = -3
b = 18958286285566608000408668544493926415504680968679321075787234672564

Gx = 19277929113566293071110308034699488026831934219452440156649784352033
Gy = 19926808758034470970197974370888749184205991990603949537637343198772
G = (Gx, Gy)

n = 26959946667150639794667015087019625940457807714424391721682722368061
h = 1
P224 = EllipticCurve(a, b, p, G, n, h)
# as specified in https://doi.org/10.6028/NIST.SP.800-186

After that, each one generates his private key(random integer) :

In [34]:
import random
prka = random.randint(2,n-1)
print("Alice prka=",prka)

prkb = random.randint(2,n-1)
print("Bob prkb=",prkb)

Alice prka= 5267097285810276234098287713795220123804508800007370965018422007444
Bob prkb= 11800237104009462717038422864898912315791792588849912235953827315360


Then, Alice and Bob generate their public keys and share them with eachother :

In [35]:
pubkA = P224.multiply(prka, G)
print("Alice pubkA=",pubkA)

pubkB = P224.multiply(prkb, G)
print("Bob pubkB=",pubkB)

Alice pubkA= (10, 9)
Bob pubkB= (15, 16)


After receiving them, Alice and Bob can calculate the secrete key by multiplying their respective private keys by the public key the received :

In [36]:
keyAB = P224.multiply(prka, pubkB)
print("keyAB=",keyAB)
keyBA = P224.multiply(prkb, pubkA)
print("keyBA=",keyBA)

keyAB= (20, 17)
keyBA= (20, 17)


We can see that Alice and Bob have the same secret key and can now use it to encrypt their messages.